In [1]:
%reset

# Imports

In [1]:
import numpy as np
import PayloadClass as Payload
import BaseClass as Base
import L4satClass as L4Sat
import MarssatClass as MarSat
from datetime import datetime
import json
import os
import pandas as pd

# Settings
- Base Location and simulation timing
- time_step
    - Set to 720 to recreate data used for analysis
    - Set to 14400 to run sim in one minute

In [ ]:
time_step = 720 #seconds per step
base_lattitude = 39.8 #n
base_longitude = 202.1 #E
start_date = "2038-04-15-06-00-00"
end_date = "2039-04-15-06-00-00"

# Import from CSV for payloads

In [3]:
folder_path = 'PayloadCSVs'
payloads_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        variable_name = os.path.splitext(filename)[0]
        csv_path = os.path.join(folder_path, filename)
        df = pd.read_csv(csv_path)
        
        # Assign to a variable with the same name as the file
        globals()[variable_name] = df
        payloads_data.append(df)
print(payloads_data)
#deep drilliing system, Payload Name Payload Manipulation Rover 1 & 2, Tansport rover 2, arm do not have any commands

[                   Payload Name          CH4 Tank 1
0                    Identifier                  14
1                     Mass (kg)  210.33499999999998
2   Max Power Consumption (kWe)               0.023
3        Power Proudction (kWe)                   0
4           Power Storage (kWh)                   0
5             Share Power (T/F)                   F
6               X Dimension (m)  2.3000000000000003
7               Y Dimension (m)  2.3000000000000003
8               Z Dimension (m)  3.7319999999999998
9            Safe Mode Duty (%)                   5
10         Sandby Mode Duty (%)                  25
11     Experiment Mode Duty (%)                 100,                    Payload Name          CH4 Tank 2
0                    Identifier                  15
1                     Mass (kg)  210.33499999999998
2   Max Power Consumption (kWe)               0.023
3        Power Proudction (kWe)                   0
4           Power Storage (kWh)                   0
5         

# Load in commands
- "Command_Sets\\Nominal.csv"
    - Used to smulate normal operations
- "Command_Sets\\Failure 1.csv"
    - Simulates losing an FSP on the 1st day
- "Command_Sets\\Failure 2.csv"
    - Simulates losing an FSP on the 30th day
- "Command_Sets\\Failure 3.csv"
    - Simulates losing an FSP on the 70th day

In [4]:
command_file = pd.read_csv("Command_Sets\\nominal.csv")
command_start = command_file[command_file['Start Time (hr elasped)'] == 0].sort_values(by='Payload ID')
display(command_start)

,Payload ID,Mode,Movement (T/F),Movement Vehicle,Final X,Final Y,Final Z,Start Time (hr elasped),Estimate Run time (hr)
0,0.0,Safe,T,Starship,0.0,0.0,0.0,0.0,50.0
7,1.0,Safe,T,Starship,0.0,0.0,0.0,0.0,50.0
14,2.0,Safe,T,Starship,0.0,0.0,0.0,0.0,2.0
18,3.0,Safe,T,Starship,0.0,0.0,0.0,0.0,2.0
22,4.0,Safe,T,Starship,0.0,0.0,0.0,0.0,2.0
46,5.0,Safe,T,Starship,0.0,0.0,0.0,0.0,2.0
70,6.0,Safe,T,Starship,0.0,0.0,0.0,0.0,48.0
77,7.0,Safe,T,Starship,0.0,0.0,0.0,0.0,56.0
83,8.0,Safe,T,Starship,0.0,0.0,0.0,0.0,48.0
89,9.0,Safe,T,Starship,0.0,0.0,0.0,0.0,26.0


# Create all classes
- Generates and initlizes the payloads, the base, and satelites

In [5]:
MATER_base = Base.Base(start_date,base_lattitude,base_longitude,0.1,500)
L4_sat = L4Sat.SunMarsL4_Relay("L4",1,1,1,1,1,1,1,1) #L4 Relay creation
Mars_sat = MarSat.MarsOrbit_Relay("mars sat",1,1,1,1,1,1,1,1,100000,0.95,90,0,-90,0,start_date)
payloads = []
for payload in payloads_data:
    name        = payload.columns[1]
    ID          = int(payload.iloc[0,1])
    mass        = float(payload.iloc[1,1])
    max_pwr     = float(payload.iloc[2,1])
    pwr_prod    = float(payload.iloc[3,1])
    share_pwr   = payload.iloc[5,1] == "F"
    pwr_store   = float(payload.iloc[4,1])
    x_d         = float(payload.iloc[6,1])
    y_d         = float(payload.iloc[7,1])
    z_d         = float(payload.iloc[8,1])
    s_duty      = float(payload.iloc[9,1])
    sb_duty     = float(payload.iloc[10,1])
    e_duty      = float(payload.iloc[11,1])
    #print(ID)
    x_p         = float(command_start.loc[command_start['Payload ID'] == ID, 'Final X'].item())
    y_p         = float(command_start.loc[command_start['Payload ID'] == ID, 'Final Y'].item())
    z_p         = float(command_start.loc[command_start['Payload ID'] == ID, 'Final Z'].item())
    loc         = command_start.loc[command_start['Payload ID'] == ID, 'Movement Vehicle'].item()
    payloads.append(Payload.Payload(name, ID, mass, max_pwr, pwr_prod, share_pwr, pwr_store, x_d, y_d, z_d, s_duty, sb_duty, e_duty, x_p, y_p, z_p, loc))
    # print(payloads[-1].get_specs())
payloads[0].get_specs()[1]
payloads = sorted(payloads, key=lambda obj: obj.get_specs()[1])
for payload in payloads:
    print(payload.get_specs())


['Deep Drilling System', 0, 34.5, 1.15, 0.0, True, 0.0, 1.25, 1.25, 3.162, 5.0, 25.0, 100.0]
['Combustion Test', 1, 812.6704999999998, 2.3, 0.0, True, 0.0, 1.07, 1.85, 0.892, 5.0, 25.0, 100.0]
['Payload Transporation Rover 1', 2, 5290.0, 1.265, 1.1, False, 14.88, 4.5, 4.26, 2.7, 5.0, 25.0, 100.0]
['Payload Transporation Rover 2', 3, 5290.0, 1.265, 1.1, False, 14.88, 4.5, 4.26, 2.7, 5.0, 25.0, 100.0]
['Payload Manipulation Rover 1', 4, 322.0, 1.7249999999999999, 1.5, False, 30.0, 1.8, 1.492, 2.062, 5.0, 25.0, 100.0]
['Payload Manipulation Rover 2', 5, 322.0, 1.7249999999999999, 1.5, False, 30.0, 1.8, 1.492, 2.062, 5.0, 25.0, 100.0]
['Extravehicular Mobility Test', 6, 74.75, 0.0552, 0.0, True, 0.0, 1.5, 1.042, 0.6900000000000001, 5.0, 25.0, 100.0]
['Rover Docking System', 7, 1492.2975, 0.11499999999999999, 0.0, True, 0.0, 3.52, 2.2, 3.552, 5.0, 25.0, 100.0]
['Radiation Experiement', 8, 1380.0, 3.8524999999999996, 0.0, True, 0.0, 4.1499999999999995, 1.75, 0.652, 5.0, 25.0, 100.0]
['Rodwel

# Displays all payload commands

In [6]:
payload_commands = []
for payload in payloads:
    ID = payload.id
    command_set = command_file[command_file['Payload ID'] == ID].sort_values(by='Start Time (hr elasped)')
    payload_commands.append(command_set)
if True:
    for payload_num in range(0,len(payloads)):
        display(payloads[payload_num].name)
        display(payload_commands[payload_num])

'Deep Drilling System'

,Payload ID,Mode,Movement (T/F),Movement Vehicle,Final X,Final Y,Final Z,Start Time (hr elasped),Estimate Run time (hr)
0,0.0,Safe,T,Starship,0.00,0.0,0.0,0.0,50.0
1,0.0,Safe,T,Transport1,11.25,12.5,0.0,50.0,1.0
2,0.0,Safe,F,NaN,NaN,NaN,NaN,51.0,4.0
3,0.0,Standby,F,NaN,NaN,NaN,NaN,55.0,3.0
4,0.0,Active,F,NaN,NaN,NaN,NaN,58.0,1460.0
5,0.0,Safe,F,NaN,NaN,NaN,NaN,1518.0,7242.0


'Combustion Test'

,Payload ID,Mode,Movement (T/F),Movement Vehicle,Final X,Final Y,Final Z,Start Time (hr elasped),Estimate Run time (hr)
7,1.0,Safe,T,Starship,0.0,0.00,0.0,0.0,50.0
8,1.0,Safe,T,Transport2,20.0,18.75,0.0,50.0,1.0
9,1.0,Safe,F,NaN,NaN,NaN,NaN,51.0,7.0
10,1.0,Standby,F,NaN,NaN,NaN,NaN,58.0,3.0
11,1.0,Active,F,NaN,NaN,NaN,NaN,61.0,473.0
12,1.0,Safe,F,NaN,NaN,NaN,NaN,534.0,8226.0


'Payload Transporation Rover 1'

...

,Payload ID,Mode,Movement (T/F),Movement Vehicle,Final X,Final Y,Final Z,Start Time (hr elasped),Estimate Run time (hr)
212,28.0,Safe,T,Starship,0.0,0.00,0.0,0.0,56.0
213,28.0,Safe,T,NaN,12.5,11.25,0.0,56.0,184.0
214,28.0,Standby,T,NaN,-100.0,-100.00,0.0,240.0,144.0
215,28.0,Active,F,NaN,NaN,NaN,NaN,384.0,192.0
216,28.0,Standby,F,NaN,NaN,NaN,NaN,576.0,48.0
217,28.0,Safe,F,NaN,NaN,NaN,NaN,624.0,8136.0


'Ingenuity-Class Helicopters 4'

,Payload ID,Mode,Movement (T/F),Movement Vehicle,Final X,Final Y,Final Z,Start Time (hr elasped),Estimate Run time (hr)
219,29.0,Safe,T,Starship,0.0,0.00,0.0,0.0,56.0
220,29.0,Safe,T,NaN,12.5,11.25,0.0,56.0,184.0
221,29.0,Standby,T,NaN,-100.0,-100.00,0.0,240.0,144.0
222,29.0,Active,F,NaN,NaN,NaN,NaN,384.0,192.0
223,29.0,Standby,F,NaN,NaN,NaN,NaN,576.0,48.0
224,29.0,Safe,F,NaN,NaN,NaN,NaN,624.0,8136.0


'Ingenuity-Class Helicopters 5'

,Payload ID,Mode,Movement (T/F),Movement Vehicle,Final X,Final Y,Final Z,Start Time (hr elasped),Estimate Run time (hr)
226,30.0,Safe,T,Starship,0.0,0.00,0.0,0.0,56.0
227,30.0,Safe,T,NaN,12.5,11.25,0.0,56.0,184.0
228,30.0,Standby,T,NaN,-100.0,-100.00,0.0,240.0,144.0
229,30.0,Active,F,NaN,NaN,NaN,NaN,384.0,192.0
230,30.0,Standby,F,NaN,NaN,NaN,NaN,576.0,48.0
231,30.0,Safe,F,NaN,NaN,NaN,NaN,624.0,8136.0


# Create logs
- These are what track all the data for the simulation

In [7]:
#create tracking arrays
Sun_position                = []
Earth_position              = []
Mars_position               = []
Mars_angle                  = []
L4_sat_position             = []
optical_comm_link_down      = []
optical_comm_link_up        = []
radio_paths                 = []
radio_links_down            = []
radio_links_up              = []
Mars_sat_position           = []
power_generate              = []
power_used                  = []
payload_postions            = []
payload_locations           = []
payload_modes               = []
payload_modes_commanded     = []
payload_command_index       = []
payload_target_pos          = []
payload_pos                 = []

# Generation
- Runs at the start of each time step
- Gets positions, power states, and next commands



In [8]:
def generation(cur_date, time):
    sun_pos                 = MATER_base.get_Sun_pos()
    earth_pos               = MATER_base.get_Earth_pos()
    mars_pos                = MATER_base.get_Mars_pos()
    mars_ang                = MATER_base.get_mars_angle()
    L4_pos                  = L4_sat.get_position(cur_date)
    mars_sat_pos            = Mars_sat.get_position(cur_date)
    power_generated         = 0 
    commands_index          = []
    payload_modes_switch    = []
    payload_target_pos      = []
    
    for i in range(0,len(payloads)):
        if payload_command_index:
            more_commands = payload_command_index[-1][i]+1 < len(payload_commands[i])
        else:
            more_commands = True
        pwr = payloads[i].consume_power(time_step)
        if pwr < 0 and payloads[i].share_pwr:
            power_generated += -pwr
        if not payload_command_index:
            commands_index.append(0)
            payload_modes_switch.append("Safe")
            payload_target_pos.append(payloads[i].get_position())

        elif more_commands and time/60/60 >= payload_commands[i].iloc[payload_command_index[-1][i]+1,7]:
            mode = payload_commands[i].iloc[payload_command_index[-1][i]+1,1]
            payload_modes_switch.append(mode)
            if str(payload_commands[i].iloc[payload_command_index[-1][i]+1,2]) == 'T':
                payload_target_pos.append([float(payload_commands[i].iloc[payload_command_index[-1][i]+1,4]),float(payload_commands[i].iloc[payload_command_index[-1][i]+1,5]),float(payload_commands[i].iloc[payload_command_index[-1][i]+1,6])])
            else:
                payload_target_pos.append(payloads[i].get_position())
            commands_index.append(payload_command_index[-1][i]+1)

        else:
            mode = payload_commands[i].iloc[payload_command_index[-1][i],1]
            payload_modes_switch.append(mode)
            payload_target_pos.append(payloads[i].get_position())
            commands_index.append(payload_command_index[-1][i])
    return sun_pos, earth_pos, mars_pos, mars_ang, L4_pos, mars_sat_pos, power_generated, payload_modes_switch, commands_index, payload_target_pos

## Production
- Main calculatioin for the loop
- Sims power producton, consumption and communicatiion links
- Moves payloads if necessary

In [9]:
def production(cur_date, time, earth_pos, mars_pos, L4_pos, mars_sat_pos, sun_pos, payload_modes_switch, payload_target_pos):
   earth_mars_dist         = MATER_base.get_earth_distance()
   earth_view              = MATER_base.get_Earth_view()
   earth_eclipse           = MATER_base.get_earth_eclipse()
   L4_mars_dist            = L4_sat.get_Mars_distance(cur_date)
   L4_earth_dist           = L4_sat.get_Earth_distance(cur_date)
   L4_view                 = MATER_base.get_L4_view()
   L4_earth_view           = not(L4_sat.get_earth_eclipse(earth_pos, sun_pos))
   mars_sat_earth_view     = Mars_sat.get_view(mars_pos, earth_pos)
   mars_sat_mars_view      = MATER_base.get_satellite_visible(mars_sat_pos)
   mars_sat_L4_view        = Mars_sat.get_view(mars_pos, L4_pos)
   mars_sat_mars_distance  = MATER_base.get_satellite_distance(mars_sat_pos)

   # Link Budget
   #Optical
   optical_comm_link_margin_down = MATER_base.link_budget()
   optical_comm_link_margin_up = MATER_base.link_budget()

   #radio
   radio_link_margin_down = []
   radio_link_margin_up = []

 
   if (earth_view and (not(earth_eclipse))):
      radio_path = "Mars - Earth"
      radio_link_margin_down.append(MATER_base.earth_link_margin_down(earth_mars_dist))
      radio_link_margin_up.append(MATER_base.earth_link_margin_up(earth_mars_dist))
   elif (mars_sat_earth_view and mars_sat_mars_view and (not(earth_eclipse))):
      radio_path = "Mars - Mars Sat - Earth"
      radio_link_margin_down.append(Mars_sat.mars_link_margin_down(mars_sat_mars_distance))
      radio_link_margin_up.append(Mars_sat.mars_link_margin_up(mars_sat_mars_distance))
      radio_link_margin_down.append(MATER_base.earth_link_margin_down(earth_mars_dist))
      radio_link_margin_up.append(MATER_base.earth_link_margin_up(earth_mars_dist))
   elif (L4_earth_view):
      if(L4_view):
         radio_path = ("Mars - L4 Sat - Earth")
         radio_link_margin_down.append(L4_sat.mars_link_margin_down(L4_mars_dist))
         radio_link_margin_up.append(L4_sat.mars_link_margin_up(L4_mars_dist))
         radio_link_margin_down.append(L4_sat.earth_link_margin_down(L4_earth_dist))
         radio_link_margin_up.append(L4_sat.earth_link_margin_up(L4_earth_dist))
      elif(mars_sat_mars_view and mars_sat_L4_view):
         radio_path = ("Mars - Mars Sat - L4 Sat - Earth")
         radio_link_margin_down.append(Mars_sat.mars_link_margin_down(mars_sat_mars_distance))
         radio_link_margin_up.append(Mars_sat.mars_link_margin_up(mars_sat_mars_distance))
         radio_link_margin_down.append(L4_sat.mars_link_margin_down(L4_mars_dist))
         radio_link_margin_up.append(L4_sat.mars_link_margin_up(L4_mars_dist))
         radio_link_margin_down.append(L4_sat.earth_link_margin_down(L4_earth_dist))
         radio_link_margin_up.append(L4_sat.earth_link_margin_up(L4_earth_dist))
      else:
         radio_link_margin_down.append(-100)
         radio_link_margin_up.append(-100)
         radio_path = "No Path"
   else:
      radio_link_margin_down.append(-100)
      radio_link_margin_up.append(-100)
      radio_path = "No Path"
   
       
   #power cosnumption 
   power_consumed = 0
   cur_mode = []
   for i in range(0,len(payloads)):
      pwr = payloads[i].consume_power(time_step)
      if pwr > 0 and payloads[i].share_pwr:
         power_consumed += pwr 
      payloads[i].set_mode(payload_modes_switch[i])
      cur_mode.append(payloads[i].get_mode())
      if payloads[i].get_position()[0] != payload_target_pos[i][0] or payloads[i].get_position()[1] != payload_target_pos[i][1]:
         payloads[i].set_position(payload_target_pos[i][0], payload_target_pos[i][1], MATER_base.get_surface())
    

   return earth_mars_dist, earth_view, L4_mars_dist, L4_earth_dist, L4_view, optical_comm_link_margin_down, optical_comm_link_margin_up,radio_link_margin_down, radio_link_margin_up, radio_path, power_consumed, cur_mode

# Output

- Appends all information to the log arrays

In [10]:
def output(sun_pos, time, earth_pos, mars_pos, mars_ang, L4_pos, optical_comm_link_margin_down,optical_comm_link_margin_up,radio_link_margin_down,radio_link_margin_up,radio_path,mars_sat_pos,power_generated,power_consumed, payload_modes_switch, commands_index, cur_mode):
    Sun_position.append(sun_pos)
    Earth_position.append(earth_pos)
    Mars_position.append(mars_pos)
    Mars_angle.append(mars_ang)
    L4_sat_position.append(L4_pos)
    optical_comm_link_down.append(optical_comm_link_margin_down)
    optical_comm_link_up.append(optical_comm_link_margin_up)
    radio_paths.append(radio_path)
    Mars_sat_position.append(mars_sat_pos)
    radio_links_down.append(radio_link_margin_down)
    radio_links_up.append(radio_link_margin_up)
    power_generate.append(power_generated)
    power_used.append(power_consumed)
    cur_positions = []
    cur_locations = []
    for payload in payloads:
        cur_positions.append(payload.get_position())
        cur_locations.append(payload.get_location())
    payload_postions.append(cur_positions)
    payload_locations.append(cur_locations)
    payload_command_index.append(commands_index)
    payload_modes.append(cur_mode)
    payload_modes_commanded.append(payload_modes_switch)

    pass

# Propagator

- Runs the above functions for all time steps

In [11]:
total_seconds = (datetime.strptime(end_date,'%Y-%m-%d-%H-%M-%S') - datetime.strptime(start_date, '%Y-%m-%d-%H-%M-%S')).total_seconds()
time = 0.
print(total_seconds)
while (time <= total_seconds):
    MATER_base.update(time_step)
    cur_date = MATER_base.get_date()

    sun_pos, earth_pos, mars_pos, mars_ang, L4_pos, mars_sat_pos, power_generated, payload_modes_switch, commands_index, payload_target_pos = generation(cur_date, time)

    earth_mars_dist, earth_view, L4_mars_dist, L4_earth_dist, L4_view, optical_comm_link_margin_down, optical_comm_link_margin_up,radio_link_margin_down, radio_link_margin_up, radio_path, power_consumed, cur_mode = production(cur_date, time, earth_pos, mars_pos, L4_pos, mars_sat_pos, sun_pos, payload_modes_switch, payload_target_pos)

    output(sun_pos, time, earth_pos, mars_pos, mars_ang, L4_pos, optical_comm_link_margin_down,optical_comm_link_margin_up,radio_link_margin_down,radio_link_margin_up,radio_path,mars_sat_pos,power_generated,power_consumed, payload_modes_switch, commands_index, cur_mode)

    print(time)
    time = time+time_step



31536000.0
0.0
14400.0
28800.0
43200.0
57600.0
72000.0
86400.0
100800.0
115200.0
129600.0
144000.0
158400.0
172800.0
187200.0
201600.0
216000.0
230400.0
244800.0
259200.0
273600.0
288000.0
302400.0
316800.0
331200.0
345600.0
360000.0
374400.0
388800.0
403200.0
417600.0
432000.0
446400.0
460800.0
475200.0
489600.0
504000.0
518400.0
532800.0
547200.0
561600.0
576000.0
590400.0
604800.0
619200.0
633600.0
648000.0
662400.0
676800.0
691200.0
705600.0
720000.0
734400.0
748800.0
763200.0
777600.0
792000.0
806400.0
820800.0
835200.0
849600.0
864000.0
878400.0
892800.0
907200.0
921600.0
936000.0
950400.0
964800.0
979200.0
993600.0
1008000.0
1022400.0
1036800.0
1051200.0
1065600.0
1080000.0
1094400.0
1108800.0
1123200.0
1137600.0
1152000.0
1166400.0
1180800.0
1195200.0
1209600.0
1224000.0
1238400.0
1252800.0
1267200.0
1281600.0
1296000.0
1310400.0
1324800.0
1339200.0
1353600.0
1368000.0
1382400.0
1396800.0
1411200.0
1425600.0
1440000.0
1454400.0
1468800.0
1483200.0
1497600.0
1512000.0
1526400.0


# Saves the data to a .json file for processing

In [13]:
data = {
    "time_step"                     : time_step,
    "Sun_position"                  : Sun_position,
    "Earth_position"                : Earth_position,
    "Mars_position"                 : Mars_position,
    "L4_sat_position"               : L4_sat_position,
    "optical_comm_link_down"        : optical_comm_link_down,
    "optical_comm_link_up"          : optical_comm_link_up,
    "radio_paths"                   : radio_paths,
    "radio_links_down"              : radio_links_down,
    "radio_links_up"                : radio_links_up,
    "Mars_sat_position"             : Mars_sat_position,
    "Power Produced"                : power_generate,
    "Power Consumed"                : power_used,
    "Payload Positions"             : payload_postions,
    "Payload Locations"             : payload_locations,
    "Payload Modes"                 : payload_modes
}

def convert_ndarray(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Type {type(obj)} not serializable")

# Save to JSON
with open("data.json", "w") as f:
    json.dump(data, f, indent=4, default=convert_ndarray)